# Multiple Linear Regression : House Prices

This lab demonstrates building a multi-linear regression ML model to predit house prices

## Step 1 - Load Data

- The first step is to load the pandas module that allows us to use dataframes. 
- We also set the number of decimal points to display
- The dataset used here is downloadable from the URL in the lab notes.
- After reading it in, we display the first five rows to see that the data loaded ok

In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression





In [2]:
dataset = 'datasets/house-sales-full.csv'
pd.options.display.float_format = '{:,.2f}'.format
house_prices = pd.read_csv(dataset)
house_prices.head()

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
0,1,9/16/14,280000,1000102,Multiplex,9/1/14,405100,0.93,"300,805.00",2,...,3.00,6,7,1991,0,0,70000,229000,98002,False
1,2,6/16/06,1000000,1200013,Single Family,6/1/06,404400,0.93,"1,076,162.00",1,...,3.75,4,10,2005,0,0,203000,590000,98166,True
2,3,1/29/07,745000,1200019,Single Family,1/1/07,425600,0.98,"761,805.00",1,...,1.75,4,8,1947,0,0,183000,275000,98166,False
3,4,2/25/08,425000,2800016,Single Family,2/1/08,418400,0.96,"442,065.00",1,...,3.75,5,7,1966,0,0,104000,229000,98168,False
4,5,3/29/13,240000,2800024,Single Family,3/1/13,351600,0.81,"297,065.00",1,...,1.75,4,7,1948,0,0,104000,205000,98168,False


## Step 2 - Explore Data

Before we do any training, we usually want to get some idea about what the data looks like. 
- It looks like there are a number of features that probably won't be input parameters
- The DocumentID which refers to a specific property id
- The Date which is probably the date of the document
- We may need to get an explanation as to what some of the columns mean like 'zhvi_idx'

The `describe` function can be used to give us some basic statistics about the data set.

We can explore questions motivated by the problem domain like:

- How many 'max' bedrooms do we have? 
- What are the min/max of 'SalePrice'?
- Do we have outliers in data?

In [10]:
house_prices.describe()


,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


#### Data Type Issues

- Notice that some of the columns were not used since they data type of that column, like 'Date' which is a string, is not numeric, which means that statistics cannot be computed for them
- There is some data which is also categorical, like property id and zip code, but are represented by numeric labels.
- The `describe()` just assumed the were interval data and compute the statistics anyway.
- Obviously the mean of the zip codes is meaningless even though it was computed

#### Correlation exploration

- We could also explore some possible correlations between inputs by using `house_prices.corr()`
- But this would produce an error since we can only do correlations between numeric values
- We could try doing correlation between sale price and some interesting parameters and see if anything pops out



In [12]:

 house_prices[['SqFtLot', 'SqFtTotLiving', 'Bedrooms']].corrwith(house_prices['SalePrice'])


SqFtLot         0.14
SqFtTotLiving   0.68
Bedrooms        0.32
dtype: float64

## Step 3 - Shape Data

- It seems clear that not every column in the data will be useful in the model. The correlation analysis might suggest the columns that seem to be more correlated with sale price
- Because this is supervised learning, we have to pull out the taget column, sales price, and use it as the label
- For a first cut at a model **=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [13]:
input_columns= ['Bedrooms', 'Bathrooms', 'SqFtTotLiving', 'SqFtLot', 'LandVal']
label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot,LandVal,SalePrice
0,6,3.00,2400,9373,70000,280000
1,4,3.75,3764,20156,203000,1000000
2,4,1.75,2060,26036,183000,745000
3,5,3.75,3200,8618,104000,425000
4,4,1.75,1720,8620,104000,240000
...,...,...,...,...,...,...
27058,2,1.75,1410,1161,147000,374000
27059,2,1.75,1410,1005,147000,374000
27060,4,1.00,1070,11170,92000,165000
27061,3,2.00,1345,6223,103000,315000


### Create the variables
- To use the regression library, we need to have an input x and set of labels y

In [14]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 5)
y.shape =  (27063, 1)


## Step 4 : Run Regression

- This is the simplest step, we just use the scikit-learn library to create a regression model using the standard defaults

In [15]:
model = LinearRegression ().fit(x,y)
model

LinearRegression()

## Step 5 : Predict

- With the model, we can see how good our predictor is by creating a set of predictions and comparing those to the actual lables.

In [16]:
predictions = model.predict(x)
predictions

array([[313206.30930952],
       [772741.67412942],
       [420944.98520074],
       ...,
       [140648.78714461],
       [276488.84419071],
       [419174.00295527]], shape=(27063, 1))

In [17]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot,LandVal,SalePrice,predicted_price
0,6,3.00,2400,9373,70000,280000,"313,206.31"
1,4,3.75,3764,20156,203000,1000000,"772,741.67"
2,4,1.75,2060,26036,183000,745000,"420,944.99"
3,5,3.75,3200,8618,104000,425000,"537,956.60"
4,4,1.75,1720,8620,104000,240000,"279,323.64"
...,...,...,...,...,...,...,...
27058,2,1.75,1410,1161,147000,374000,"364,174.50"
27059,2,1.75,1410,1005,147000,374000,"364,156.80"
27060,4,1.00,1070,11170,92000,165000,"140,648.79"
27061,3,2.00,1345,6223,103000,315000,"276,488.84"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

- The coefficients are a measure of how much that input affects the outcome
- A coefficient near zero means that feature has littel effect on the model
- In this data, the data is not scaled so we may see some unusual values
- The model would be better if we normalized the data

**Q==> Also inspect R2 value.  Is it decent?**

- The R² value (coefficient of determination) measures how well the independent variables (X) explain the variability in the dependent variable (Y).
- R2=1 → Perfect fit (Model explains all variance)
- R2=0 → No explanatory power (Model is no better than guessing)

In [18]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

print ("coefficients: " , model.coef_)
print ("intercept : ", model.intercept_)

coefficients:  [[-39286.34  45993.71    139.14      0.11      1.17]]
intercept :  [-5881.37]


In [19]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "coefficient": model.coef_[0]})
coef

,input_column,coefficient
0,Bedrooms,"-39,286.34"
1,Bathrooms,"45,993.71"
2,SqFtTotLiving,139.14
3,SqFtLot,0.11
4,LandVal,1.17


In [21]:
from sklearn.metrics import  r2_score


print ("R2 : " , r2_score(y, predictions))


R2 :  0.7618151215674123


## Step 7 : Predict on New Data

- Now generate new data and use the model for predictions

In [24]:
new_data = pd.DataFrame({'Bedrooms' : [5,3,2],
                         'Bathrooms' : [3,2,1.5],
                         'SqFtTotLiving' : [4400, 1800, 1550],
                         'SqFtLot' : [10000, 5000, 4000],
                         'LandVal' : [150000, 80000, 20000]
             })
new_data

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot,LandVal
0,5,3.00,4400,10000,150000
1,3,2.00,1800,5000,80000
2,2,1.50,1550,4000,20000


In [25]:
new_prediction = model.predict(new_data)
new_prediction

array([[724355.59],
       [312777.54],
       [224038.85]])

In [26]:
pd.options.display.float_format = '{:,.2f}'.format

new_data['predicted_price'] = new_prediction
new_data

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot,LandVal,predicted_price
0,5,3.00,4400,10000,150000,"724,355.59"
1,3,2.00,1800,5000,80000,"312,777.54"
2,2,1.50,1550,4000,20000,"224,038.85"


## Step 8 : Impo]rove Model Performance

- Now that we have done a 'end-to-end' implementation in Regression.  
- How ever our accuracy isn't all that great!  

**Q ==> What can we do to improve our model?**

- One option is to choose better or more input columns.  
- Examine the data and suggest other columns
- The data could be scaled so that all parameters are in the range [0,1]
- We could convert strings like dates and booleans like new construction into numerics to see if they have an effect
